# April 2019 Python Users Group - Manchester, New Hampshire, USA

Author: Larry M. (Larz60+)
***

***
##  MIT License

Copyright (c) 2018 Larz60+

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

***
##  Introduction


Originally, I wanted to present examples of three types of scrapers, one using lxml, another using BeautifulSoup and finally one using Selenium. I don't think I could present all three in one evening and still have good coverage of each section, so I'll leave lxml for another presentation. Instead, I'll be scraping a site that requires both selenium and BeautifulSoup. A good candidate for this is the Connecticut Secretary of State Business Entity Search site. 

The modules needed to accomplish this are:

### Set-up

* CreateCityFile.py - Create city dictionary, saved as json file by scraping 'https://ctstatelibrary.org/cttowns/counties' using selenium and BeautifulSoup. Will
  explain how to capture XPath using Inspect Element. Page will be captured using Selenium which will expand the java script. The parsing will be done using
  Beautifulsoup.

* CreateDict.py - Module to create class for creating dictionaries, add nodes, and add cells. (reusable)

* Copy PrettifyPage.py from other application- used to create formatted pretty pages (uses BeautifulSoup4 prettify, but modifies spacing to make it easier to read.

* BusinessPaths.py - Create a Paths class, that contains entries for and creates directories (if necessary) for all modules in package. Included are all file
  names and locations. Having a file like this saves a lot of time trying to chase down where a particular file is located. It is used by all modules, and
  because of this, any file name and/or directory can be modified, and immediately propagated to all modules in package. Uses pathlib.

### Analysis

* PreviewSearchPage.py - Runs a test using Seleium to see if there's a hidden API somewhere, which would greatly simplify scraping.

### Scrape

* ScrapeConnecticut.py - Capture all summary pages searching by City, which is the best method for getting total coverage of all businesses.
  Following done using concurrentfutures.ProcessPoolExecutor. These files are stored in ./data/html and named City_pagex.html example: Andover_page12.html

* ExtractBusinessEntities.py - Extracts business information from Summary files, and saves as preliminary information in ./data/json/CompanyMaster.json includes
  basic company information as well as URL of Detail information, and associated filename.

* AddEntityDetail.py - Using information gathered in ExtractBusinessEntities.py, downloads all detail files, and adds detail information on each business to the
  ./data/json/CompanyMaster.json file.

### Process

* CreateDatabase.py - Creates an sqlite3 database located at ./data/database/CompanyMaster.db. Populated from CompanyMain.json, CompanyDetail.json,
  CompanyPrincipals.json, CompanyAgents.json and CompanyFilings.json whics are generated from CompanyMaster.json. Calls CreateTables.py

* CreateTables.py - Creates database and all tables in Company sqlite3 database
  
* Future - Create GUI to access scraped data. (Might actually have something by meeting time).

* The website used to scrape above business data is in process of change. Will need to rewrite all modules in Scrape section.


In [ ]:
%load_ext autoreload
%autoreload 2

***
## BusinessPaths.py

This module creates all data directories (only if they don't already exist), filenames and paths (once in a while I will create tmp files on the fly, but will
almost always use paths from this module). I have been doing this for some time now and have never regretted it. Advantages:

* Can add new files, which instantly become available to every program in the package.
* Never have to wonder where a file is stored, as only it's name is necessary for access.
* Entire directory can be renamed with 100 % transparency.
* Most of the directory names become standard to all the code that I write, and users know where to look for a certain type of file.

Probably many other advantages that I can't think of at the moment.

The very first statement in each and every one of these files is an anchor that establishes the relationship of all files and directorys, and the modules themselves.
The anchor is based of the __file__ variables associated with a module, and is written as:
os.chdir( os.path.abspath( os.path.dirname( __file__ )))

Initializes all paths, URLs and files (other that temporary files). It is imported into all modules

Create .data directory and all sub directories if not already created, does not destroy existing directories or data. Safe to run.

Script should be run by itself once on a new system install.


In [ ]:
# BusinessPaths.py

from pathlib import Path
import os


class BusinessPaths:
    def __init__(self):
        os.chdir(os.path.abspath(os.path.dirname(__file__)))
        self.homepath = Path('.')
        self.rootpath = self.homepath / '..'

        self.datapath = self.rootpath / 'data'
        self.datapath.mkdir(exist_ok=True)

        self.dbpath = self.datapath / 'database'
        self.dbpath.mkdir(exist_ok=True)

        self.htmlpath = self.datapath / 'html'
        self.htmlpath.mkdir(exist_ok=True)

        self.idpath = self.datapath / 'Idfiles'
        self.idpath.mkdir(exist_ok=True)
        
        self.jsonpath = self.datapath / 'json'
        self.jsonpath.mkdir(exist_ok=True)

        self.prettypath = self.datapath / 'pretty'
        self.prettypath.mkdir(exist_ok=True)

        self.textpath = self.datapath / 'text'
        self.textpath.mkdir(exist_ok=True)

        self.tmppath = self.datapath / 'tmp'
        self.tmppath.mkdir(exist_ok=True)

        self.base_url = 'http://searchctbusiness.ctdata.org/'

        self.cities_json = self.jsonpath / 'cities.json'
        self.city_list_url = 'https://ctstatelibrary.org/cttowns/counties'
        self.raw_city_file = self.tmppath / 'raw_city.html'
        # self.cities_text = self.textpath / 'cities.txt'

        self.company_master_json = self.jsonpath / 'CompanyMaster.json'

        self.CompanyMasterDb = self.dbpath / 'CompanyMaster.db'

        self.company_main = self.jsonpath / 'CompanyMain.json'
        self.company_detail = self.jsonpath / 'CompanyDetail.json'
        self.company_principals = self.jsonpath / 'CompanyPrincipals.json'
        self.company_agents = self.jsonpath / 'CompanyAgents.json'
        self.company_filings = self.jsonpath / 'CompanyFilings.json'


if __name__ == '__main__':
    BusinessPaths()


***
## CreateDict.py

I got tired of trying to remember exactly how (and where) to add new dictionaries, nodes or cells, so I wrote this module.

### new_dict(self, dictname)
* Create a new dictionary instance named dictname. I usually manually create the dictionary in the __init__ method of class, but this can be used as an alternative

### add_node(self, parent, nodename)
* Adds a new nested node to parent node, named nodename

### Add cell(self, nodename, cellname, value)
* Adds a cell to node nodename, named cellname, with and assigns value to cell.

### display_dict(self, dictname, level=0)
* Display dictionary (or node of dictionary) in a nicely formatted and properly indented manner.
* level is indent level, and never supplied by caller, it's used for determining indent level for recurrsive calls.
* testit is a demo of usage.
  - Results of running testit:
  - CityList Dictionary

    Boston

        Resturants

            Spoke Wine Bar
                Addr1: 02144
                City: Sommerville
                Phone: 617-718-9463

            Highland Kitchen
                Addr1: 150 Highland Ave
                City: Sommerville
                ZipCode: 02144
                Phone: 617-625-1131

    raw data: {'Boston': {'Resturants': {'Spoke Wine Bar': {'Addr1': '02144', 'City': 'Sommerville', 'Phone': '617-718-9463'}, 'Highland Kitchen': {'Addr1': '150 Highland Ave', 'City': 'Sommerville', 'ZipCode': '02144', 'Phone': '617-625-1131'}}}}

In [ ]:
# CreateDict.py

import os


class CreateDict:
    def __init__(self):
        os.chdir(os.path.abspath(os.path.dirname(__file__)))

    def new_dict(self, dictname):
        setattr(self, dictname, {})

    def add_node(self, parent, nodename):
        node = parent[nodename] = {}
        return node

    def add_cell(self, nodename, cellname, value):
        cell =  nodename[cellname] = value
        return cell

    def display_dict(self, dictname, level=0):
        indent = " " * (4 * level)
        for key, value in dictname.items():
            if isinstance(value, dict):
                print(f'\n{indent}{key}')
                level += 1
                self.display_dict(value, level)
            else:
                print(f'{indent}{key}: {value}')
            if level > 0:
                level -= 1


def testit():
    # instantiate class
    cd = CreateDict()

    # create new dictionary named CityList
    cd.new_dict('CityList')

    # add node Boston
    boston = cd.add_node(cd.CityList, 'Boston')
    # add sub node Resturants
    bos_resturants = cd.add_node(boston, 'Resturants')

    # Add subnode 'Spoke Wine Bar' to parent bos_resturants
    spoke = cd.add_node(bos_resturants, 'Spoke Wine Bar')
    cd.add_cell(spoke, 'Addr1', '89 Holland St')
    cd.add_cell(spoke, 'City', 'Sommerville')
    cd.add_cell(spoke, 'Addr1', '02144')
    cd.add_cell(spoke, 'Phone', '617-718-9463')

    # Add subnode 'Highland Kitchen' to parent bos_resturants
    highland = cd.add_node(bos_resturants, 'Highland Kitchen')
    cd.add_cell(highland, 'Addr1', '150 Highland Ave')
    cd.add_cell(highland, 'City', 'Sommerville')
    cd.add_cell(highland, 'ZipCode', '02144')
    cd.add_cell(highland, 'Phone', '617-625-1131')

    # display dictionary
    print(f'\nCityList Dictionary')
    cd.display_dict(cd.CityList)
    print(f'\nraw data: {cd.CityList}')

if __name__ == '__main__':
    testit()


***
## CreateCityFile.py

This script creates a json file that will be used to control the order and scope of page retrevial.

This method was chosen over search by name Alphabetically because:
* The Connecticut search by Name requires a minimum or two letters
* It doesn't care where within the name those two letters occur 
* This means a search for 'AA' will find
  'VEREIT AA STRATFORD CT, LLC'
  'AA CLEANING LLC'
  but not 'AAA AFFORDABLE GLASS, INC.'
* because of this, the number of pages needed for full coverage
  is massive, and probably impossible because of practability.
  
Coverage by city has no page limit (at time of writing)
* example a city search on bridgeport brings up 741 pages.
* This allows for full coverage of all companies.

The URL I chose to get city information is https://ctstatelibrary.org/cttowns/counties. 
This page contains more information than is needed for this package.
Nevertheless, I'll save everything ... It may with  high probability, prove useful in some future package.

To start, capture the city page using selenium. This will capture the javascript city table in the browser page.
* Show Inispect Element to get css select for header
* Show how to determine table layout using print statements
* Then uncomment code and run again, showing the dictionary that's written to the json file (data/json/cities.json).
* Also demonstrate use of pythomn json formatter to create text file of dictionary

Each city has the following format:
* "Andover": {
  - "Town name": "Andover",
  - "County": "Tolland",
  - "Year Established": "1848",
  - "Parent Town": "Coventry, Hebron",
  - "History of incorporation": "May 18, 1848; taken from Hebron and Coventry",
  - "ContiguousCityName": "Andover"
}


In [ ]:
# CreateCityFile.py

from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import BusinessPaths
import time
import PrettifyPage
import CreateDict
import json
import sys


class CreateCityFile:
    def __init__(self):
        self.bpath = BusinessPaths.BusinessPaths()
        self.pp = PrettifyPage.PrettifyPage()
        self.cd = CreateDict.CreateDict()

        self.header = []
        self.city_info = {}

        self.get_city_info()

    def start_browser(self):
        caps = webdriver.DesiredCapabilities().FIREFOX
        caps["marionette"] = True
        self.browser = webdriver.Firefox(capabilities=caps)

    def stop_browser(self):
        self.browser.close()

    def get_city_info(self):
        if not self.bpath.cities_json.exists():
            if self.bpath.raw_city_file.exists():
                with self.bpath.raw_city_file.open() as fp:
                    page = fp.read()
                soup = BeautifulSoup(page, "lxml")
            else:
                self.start_browser()
                self.browser.get(self.bpath.city_list_url)
                time.sleep(2)
                page = self.browser.page_source
                # save working copy and pretty copy for analysis in temp
                with self.bpath.raw_city_file.open('w') as fp:
                    fp.write(page)
                soup = BeautifulSoup(page, "lxml")
                prettyfile = self.bpath.prettypath / 'raw_city_file_pretty.html'
                with prettyfile.open('w') as fp:
                    fp.write(f'{self.pp.prettify(soup, 2)}')
                self.stop_browser()
            table = soup.find('table', {'summary': 'This table displays Connecticut towns and the year of their establishment.'})
            trs = table.tbody.find_all('tr')

            # Create Node to separate Connecticut - May contain multiple states later
            masternode = self.cd.add_node(self.city_info, 'Connecticut')
            
            citynode = None
            contigname = 'Unspecified'
            for n, tr in enumerate(trs):
                if n == 0:
                    self.header = []
                    for td in self.get_td(tr):                            
                        self.header.append(td.p.b.i.text.strip())
                    self.cd.add_cell(masternode, 'Header', self.header)
                else:
                    for n1, td in enumerate(self.get_td(tr)):
                        # print(f'==================================== tr {n}, td: {n1} ====================================')
                        # print(f'{self.pp.prettify(td, 2)}')
                        if n1 == 0:
                            citynode = self.cd.add_node(masternode, f'{td.p.text.strip()}')
                        value = td.p
                        if td.p is None:
                            value = 'Unspecified'
                        else:
                            value = td.p.text.strip()
                            if value == '—-':
                                value ='No parent town'
                        self.cd.add_cell(citynode, self.header[n1], value.strip())
                        if self.header[n1] == 'Town name':
                            contigname = value.strip().replace(' ', '')
                    self.cd.add_cell(citynode, 'ContiguousCityName', contigname)
            self.cd.display_dict(self.city_info)

            # Create json file
            with self.bpath.cities_json.open('w') as fp:
                json.dump(self.city_info, fp)


    def get_td(self, tr):
        tds = tr.find_all('td')
        for td in tds:
            yield td

if __name__ == '__main__':
    CreateCityFile()


In [4]:
from IPython.display import IFrame
IFrame(src='http://searchctbusiness.ctdata.org/', width=1450, height=900)

***
## PreviewSearchPage.py

Take a look at the search page above. At this point in time, it's the go to page for any Business Entity Search for Connecticut.

Connecticut is considering publishing an API for Business Entity search, but does not yet do that.

I have found that many (if not most) searches do indeed have an underlying API. It only has to be discovered.

The purpose of this script is to expose that API if indeed it does exist.
The method used to do this is to scrape a page using Selenium, using a city that is known to contain several pages of data.

What is garnered here?

* URL for Main page, just as a warm and fuzzy (makes sure I am using proper and full URL)
* URL for initial search page for selected city.
* URL for 2nd page for selected city
* URL for Detail information of just one company.

If there is an API, it should be exposed by step 3.
But need to see step 2 results to see if first page needs to be treated in a special way.

In [ ]:
# PreviewSerchPage.py

from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import BusinessPaths
import time
import PrettifyPage
import GetPage
import CreateDict
import json
import sys


class PreviewSearchPage:
    def __init__(self):
        self.bpath = BusinessPaths.BusinessPaths()
        self.pp = PrettifyPage.PrettifyPage()
        self.cd = CreateDict.CreateDict()
        self.gp = GetPage.GetPage()
        self.getpage = self.gp.get_page

        self.analyze_page()

    def start_browser(self):
        caps = webdriver.DesiredCapabilities().FIREFOX
        caps["marionette"] = True
        self.browser = webdriver.Firefox(capabilities=caps)

    def stop_browser(self):
        self.browser.close()

    def save_page(self, filename):
        soup = BeautifulSoup(self.browser.page_source, "lxml")
        with filename.open('w') as fp:
            fp.write(self.pp.prettify(soup, 2))
    
    def analyze_page(self):
        self.start_browser()
        self.get_search_page('Andover')
        self.stop_browser()
    
    def get_search_page(self, searchitem):
        # pick city with multiple pages
        url = 'http://searchctbusiness.ctdata.org'
        
        # Get the main page
        self.browser.get(url)
        time.sleep(2)
        
        # Even though we already know what the URL for first page is, display captured URL
        # in order to avoid any surprises.
        print(f'Main Page URL: {self.browser.current_url}')

        # Find the 'Business City' option and select it
        self.browser.find_element(By.XPATH, '/html/body/div[2]/div[4]/div/form/div/div/span[1]/select/option[3]').click()
        
        # Finde the search box, make sure it's cleared, and insert our test page 'Andover'
        searchbox = self.browser.find_element(By.XPATH, '//*[@id="query"]')
        searchbox.clear()
        searchbox.send_keys(searchitem)
        
        # Find the select button and click it to go to First page
        self.browser.find_element(By.XPATH, '/html/body/div[2]/div[4]/div/form/div/div/span[3]/button').click()
        time.sleep(2)
        
        # Display first results page URL
        print(f'Results Page 1 URL: {self.browser.current_url}')
        
        # find total number of pages, we don't need it now, but can start thinking about how to use this to 
        # help with page navigation later on when scraping all pages.
        pages = self.browser.find_element(By.XPATH, '/html/body/div[2]/div/div[2]/div[3]/div[2]/div/span[2]')
        print(pages.get_attribute('innerHTML'))
        
        
        # get page 2 by locating next page button and clickcking
        self.browser.find_element(By.XPATH, '/html/body/div[2]/div/div[2]/div[3]/div[2]/div/span[1]/a/icon').click()
        time.sleep(2)
        page2savefile = self.bpath.prettypath / 'Page2Source.html'
        self.save_page(page2savefile)
        
        # Display second results page URL
        print(f'Results Page 2 URL: {self.browser.current_url}')
        
        # Get detail page for first company in list
        self.browser.find_element(By.XPATH, '/html/body/div[2]/div/div[2]/table/tbody/tr[1]/td[1]/a').click()
        time.sleep(2)
        
        # Display detail page URL
        print(f'Detail Page URL: {self.browser.current_url}')

if __name__ == '__main__':
    PreviewSearchPage()


You can clearly see by the URL's that are gathered from the various pages that an API does exist. Although we don't have any documentation for this API (bacause it's not published)
We can break down the fields to discover how to get all the pages for a city.

Here's what was gathered:
* Main Page URL: http://searchctbusiness.ctdata.org/
* Results Page 1 URL: http://searchctbusiness.ctdata.org/search_results?query=Andover&index_field=place_of_business_city&sort_by=nm_name&sort_order=asc&page=1
* 1 of 18
* Results Page 2 URL: http://searchctbusiness.ctdata.org/search_results?page=2&start_date=1900-01-01&end_date=2019-04-01&query=Andover&index_field=place_of_business_city&sort_by=nm_name&sort_order=asc
* Detail Page URL: http://searchctbusiness.ctdata.org/business/0589152

The interesting thing here is the inconsistancy between page 1 and page 2, page 2 exposes much more of the API
So if you break down the page 2 url, here are the components:

* base url: http://searchctbusiness.ctdata.org/
* page selector: search_results?page=2
* &start_date=1900-01-01
* &end_date=2019-04-01
* &query=Andover
* &index_field=place_of_business_city
* &sort_by=nm_name
* &sort_order=asc

Pretty straight forward, can't get all options here but that's enough.
Can try page 2 url modified for page 18 (last page)

In [5]:
from IPython.display import IFrame
IFrame(src='http://searchctbusiness.ctdata.org/search_results?page=18&start_date=1900-01-01&end_date=2019-04-01&query=Andover&index_field=place_of_business_city&sort_by=nm_name&sort_order=asc', width=1450, height=900)

And It works just fine.
I also tried it with page 1 and it works.
So now this can be used to scrap all pages using requests and beautifulsoup4.

***
## ScrapeConnecticutBusiness.py

The actual scraping Script.

Note the module get_url uses the results created by PreviewSerchPage.py to create API access to Summary Page, this allows
downlaoding all of the summary files using requests rather than selenium, which is much faster.

The steps are:

### Setup

* Create a list (self.citylist) with each entry being a sublist containing City Name, filename where results will be saved, and URL. This will be used to speed up
  The feed for Concurrent Futures.
* Set a counter to show progress (self.pages_to_download)
* An empty list (self.pages_to_download) to save future results (number of summary pages for given city) in.

### Initial scrape

* Using (method: getnumpages) concurrent.futures.ProcessPoolExecutor (pure multiprocessing) fetch page1 for each city in self.citylist,

  -  Each process calls parse method, passing city, filename, and url
  -  Parse caches web pages when downloaded, the cache is checked for particular file, and file is loaded from cache if it exists. Otherwise, it is downloaded using requests.
  -  The page is parsed using BeautifulSoup, extracting paginate-info. It finishes by returning city and numpages.
  -  Use concurrent.futures.as_completed to capture results from parse (city and numpage)

### Second to final scrape

* After getting the first page of each city, it is necessary to use numpages to add entries to self.citylist so the remaining pages can be downloaded. This is the job of method add_new_pages.
  Note that the method is called from a loop in the dispacth method, in groups separated by letter of the alphabet. This allows a break for the server in Connecticut, but mostly for diabolical reasons.

  - It may seem as if this entire process could be done in the get_numpages method, but I intentionally prepair the list so that the multiprocessing part of the program doesn't contain any bottlenecks.
  - Starting with page 2, create a new self.citylist for all remaining pages or each city.
  - Call getnumpages for each group

### Notes

This entire process is blazingly fast, please note that max_workers should be changed for number of cores available in your CPU + 1. I add one, because for some reason, right or wrong, I get the feeling the script process will share one of the cores as a thread.
  

In [ ]:
    def get_url(self, city, page='1'):
        return f'{self.bpath.base_url}search_results?page={page}' \
            f'&start_date=1900-01-01&end_date=2019-04-01&query={city.upper()}' \
            f'&index_field=place_of_business_city&sort_by=nm_name&sort_order=asc'

In [ ]:
# ScrapeConnecticut.py

import BusinessPaths
import concurrent.futures
from pathlib import Path
from bs4 import BeautifulSoup
import requests
import string
import time
import json
import os
import sys
class ScrapeConnecticut:
    def __init__(self):
        self.bpath = BusinessPaths.BusinessPaths()

        self.citydict = {}
        with self.bpath.cities_json.open() as fp:
            self.citydict = json.load(fp)

        self.citylist = []
        for city, info in self.citydict['Connecticut'].items():
            if city == 'Header':
                continue
            cityn = city.replace(' ', '')
            self.citylist.append([city, self.bpath.htmlpath / f'{cityn}_page1.html', self.get_url(city)])
        
        self.pages_to_download = len(self.citylist)

        self.numpages = []
        # self.test_parse()
        self.dispatch()
        
    def get_url(self, city, page='1'):
        return f'{self.bpath.base_url}search_results?page={page}' \
            f'&start_date=1900-01-01&end_date=2019-04-01&query={city.upper()}' \
            f'&index_field=place_of_business_city&sort_by=nm_name&sort_order=asc'

    def dispatch(self):
        self.get_numpages()
        for letter in string.ascii_uppercase:
            self.add_new_pages(letter)
            self.get_numpages()

    def add_new_pages(self, letter):
        self.citylist = []
        findstart = True
        for city, npages in self.numpages:
            if findstart and not city.startswith(letter):
                print(f'findstart city: {city}')
                continue
            findstart = False
            if not city.startswith(letter):
                print(f'for break, city: {city}')
                break
            # Remove spaces from city name.
            cityn = city.replace(' ', '')

            currentpage = 2
            finalpage = int(npages)

            while True:
                # Check if done
                if currentpage > finalpage:
                    break
                url = self.get_url(city, page=str(currentpage))

                entry = [ city, self.bpath.htmlpath / f'{cityn}_page{currentpage}.html', url ]
                self.citylist.append(entry)
                currentpage += 1

        self.pages_to_download = len(self.citylist)

        # for debugging:
        # filename = self.bpath.tmppath / 'citylist.text'
        # with filename.open('w') as fp:
        #     for entry in self.citylist:
        #         fp.write(f'{entry}\n')

        print(f'Length citylist for {city}: {self.pages_to_download}')

    def parse(self, city, filename, url):
        # Will skip files already downloaded
        # print(f'city: {city}, filename: {filename}, url: {url}')        
        print(f'fetching {filename.name}')

        if filename.exists():
            with filename.open('rb') as fp:
                page = fp.read()
        else:
            response = requests.get(url)
            if response.status_code == 200:
                time.sleep(.25)
                page = response.content
                with filename.open('wb') as fp:                
                    fp.write(page)
            else:
                print("can;t download: {url}")
                return False
        soup = BeautifulSoup(page, 'lxml')
        numpages = soup.find('span', {'class': "paginate-info"}).text.split()[2]
        return city, numpages

    def test_parse(self):
        for city, filename, url in self.citylist:
            city, numpages = self.parse(city, filename, url)
            print(f'{city}: {numpages}')

    def get_numpages(self):
        countdown = self.pages_to_download
        with concurrent.futures.ProcessPoolExecutor(max_workers=5) as executor:
            city_pages = [ executor.submit(self.parse, city, filename, url) for city, filename, url in self.citylist ]
            for future in concurrent.futures.as_completed(city_pages):
                countdown -= 1
                try:
                    city, numpages = future.result()
                    print(f'{city}: {numpages}')
                except TypeError as exc:
                    print(f'TypeError exception: {exc}')
                else:
                    self.numpages.append([city, numpages])
                    print(f'Remaining files: {countdown}')


if __name__ == '__main__':
    ScrapeConnecticut()




***
## ExtractBusinessEntities.py

This module extracts Business information from the files gathered by ScrapeConnecticut.py.
It creates an intermediate dictionary which is saved as the JSON file /data/json/CompanyMaster.json
Each cell of the dictionary contains the following information:

* "0667703": {
  - "BusinessId": "0667703",
  - "BusinessName": "119 SPENCER STREET, L.L.C.",
  - "DetailUrl": "http://searchctbusiness.ctdata.org/business/0667703",
  - "Filename": "../data/Idfiles/Id0667703.html",
  - "DateFormed": "12/05/2000",
  - "Status": "Active",
  - "Address": "218 LAKE ROAD, ANDOVER, CT 06232",
  - "CityState": "ANDOVER, CT",
  - "PrincipalNames": "EDWARD A. HATEM",
  - "AgentNames": "EDWARD A. HATEM"
  - }

Key is BusinessId (also included in body for use later when creating SQL database).
Other fields are self explainatory.

### Setup

* Create a summary file list (self.summary_file_list) which is just a file list form /data/html it uses 'page' from file name to identify proper files.

### extract_company_info(self)

* This method extracts the business data form each file using BeautifulSoup. For each file in summary_file_list:

  - read file into variable page
  - Convert to soup, using 'lxml' parser
  - find tag 'table' which contains all of the business information
  - find all 'th' tags which contain the header
  - Replace various items in each header element (see self.header_replacepairs)
  - save in self.header list
  - Extract all 'tr' tags and pass to method strip_business

### strip_business(self, trs)
* continuation of extract_company_info pulls data from table tr tags

  - for each element in trs:
    - extract all td tags.
    - if first td tag:
      - there is a link to the detail page, extract that
      - Extract the BusinessId from the URL
      - Extract business name from link text
      - create the filename where detail information will be saved
      - save all this to the BusinessInfo dictionary
    - Otherwise:
      - extract td text, match to header and save to BusinessInfo dictionary

### Save BusinessInfo dictionary as json file in data/jaon/CompanyMaster.json


In [ ]:
# ExtractBusinessEntities.py

import BusinessPaths
from pathlib import Path
from bs4 import BeautifulSoup
import PrettifyPage
import CreateDict
import concurrent.futures
import requests
import string
import time
import json
import os
import sys


class ExtractBusinessEntities:
    def __init__(self):
        self.bpath = BusinessPaths.BusinessPaths()
        self.pp = PrettifyPage.PrettifyPage()
        self.cd = CreateDict.CreateDict()

        self.summary_file_list = []
        self.BusinessInfo = {}
        self.city_node = {}
        self.current_node = {}

        self.header_replacepairs = [ ('(', ''), (')', ''), ('/', ''), (' ', ''),
            ('MMDDYYYY', '') ]

        self.header = []
        self.create_business_info()

    def create_business_info(self):
        self.get_summary_file_list()
        self.extract_company_info()
        self.save_as_json()

    # |++++++|+++++++++|+++++++++| Section 1 - Setup |+++++++++|+++++++++|+++++++++|

    def get_summary_file_list(self):
        path = self.bpath.htmlpath
        self.summary_file_list = \
            [ filename for filename in path.iterdir() if filename.is_file() \
            and 'page' in filename.stem ]
        self.summary_file_list.sort()

    # |++++++|+++++++++|+++ Section 2 - Parse Summary Pages +++|+++++++++|+++++++++|

    def extract_company_info(self):
        self.header = []

        for file in self.summary_file_list:            
            print(f'Processing {file.name}')

            city = str(file.stem).split('_')[0]
            with file.open('rb') as fp:
                page = fp.read()
            soup = BeautifulSoup(page, 'lxml')

            table = soup.find('table')
            head = table.thead.find_all('th')

            for element in head:
                if not len(element):
                    continue
                item = element.text.strip()

                for a, b in self.header_replacepairs:
                    item = item.replace(a, b)
                self.header.append(item)

            trs = table.tbody.find_all('tr')
            self.strip_business(trs)

    def strip_business(self, trs):
        base_url = 'http://searchctbusiness.ctdata.org'

        for tr in trs:
            tds = tr.find_all('td')

            for n1, td in enumerate(tds):
                if n1 == 0:
                    detail_url = f"{base_url}{td.a.get('href')}"
                    business_id = detail_url.split('/')[-1]                    
                    business_name = td.a.text.strip()
                    detail_filename = self.bpath.idpath / f'Id{business_id}.html'

                    self.current_node = self.cd.add_node(self.BusinessInfo, business_id)
                    self.cd.add_cell(self.current_node, 'BusinessId', business_id)
                    self.cd.add_cell(self.current_node, 'BusinessName', business_name)
                    self.cd.add_cell(self.current_node, 'DetailUrl', detail_url)
                    self.cd.add_cell(self.current_node, 'Filename', os.fspath(detail_filename))
                else:
                    self.cd.add_cell(self.current_node, self.header[n1], 
                        td.text.strip())            

    # |++++++|+++++++++| Section 3 - Get and Parse Detail Pages +++++++++|+++++++++|

    def save_as_json(self):
        with self.bpath.company_master_json.open('w') as fp:
            json.dump(self.BusinessInfo, fp)

if __name__ == '__main__':
    ExtractBusinessEntities()


***
## AddEntityDetail.py

This module does all of the remaining scraping work. It uses the information that has already been stored in CompanyMaster.json to download and parse the detail company information.
All if the files downloaded are sroted in the /data/Idfiles directory, and each bears the filename Idxxxxxxx.html where xxxxxxx is the BusinessId. There are close to 400,000 files in this group.

The process uses concurrent.futures.ProcessPoolExecutor, since I started this project, download speed has improved greatly, at first I was getting times of 1.3 seconds per page, but at some point, I believe someone at the database location added an index that greatly increased the spped of download, now about .125 seconds or less per page.

### method add_entity_detail(self)

This method controls the entire process, from setup, download, parse and re-writing the updated CompanyMaster.json file.

when done, each entry in the master will contain, the main information gathered by ExtractBusinessEntities.py, with the addition of additional company detail, principal information, agent information, and a list of all recorded filings.

Here's what a typical entry looks like:

* "1169720": {
  - "BusinessId": "1169720",
  - "BusinessName": "182 WHEELING ROAD, LLC",
  - "DetailUrl": "http://searchctbusiness.ctdata.org/business/1169720",
  - "Filename": "../data/Idfiles/Id1169720.html",
  - "DateFormed": "03/10/2015",
  - "Status": "Active",
  - "Address": "182 WHEELING ROAD, ANDOVER, CT 06232",
  - "CityState": "ANDOVER, CT",
  - "PrincipalNames": "HEATHER L. MORTIMER",
  - "AgentNames": "GLENN T. TERK, ESQ.",
  - "BusinessDetail": {
    - "BusinessName": "182 WHEELING ROAD, LLC",
    - "CitizenshipStateInc": "Domestic / CT",
    - "LatestReport": "No Reports Found",
    - "BusinessAddress": "182 WHEELING ROAD, ANDOVER, CT 06232",
    - "BusinessType": "Unspecified",
    - "MailingAddress": "C/O LAZ PARKING, 15 LEWIS STREET, HARTFORD, CT 06103",
    - "BusinessStatus": "Active",
    - "DateIncRegistration": "Mar 10, 2015",
    - "Unused": "Unspecified"
  - },
  - "PrincipalsDetail": {
    - "NameTitle": "HEATHER L. MORTIMER MANAGER",
    - "BusinessAddress": "C/O LAZ PARKING, 15 LEWIS STREET, HARTFORD, CT 06103",
    - "ResidenceAddress": "28 JONES STREET, AMSTON, CT 06231"
  - },
  - "AgentDetail": {
    - "AgentName": "GLENN T. TERK, ESQ.",
    - "AgentBusinessAddress": "15 LEWIS STREET, HARTFORD, CT 06103",
    - "AgentResidenceAddress": "449 OLD RESERVOIR ROAD, WETHERSFIELD, CT 06109"
  - },
  - "FilingsDetail": {
    - "1": {}, This is a harmless software feature, so called because I didn't have enough time to find what causes it.
    - "2": {
      - "FilingID": "0005294665",
      - "FilingType": "ORGANIZATION",
      - "DateofFiling": "03/10/2015",
      - "VolumeType": "B",
      - "Volume": "02044",
      - "StartPage": "0685",
    - "Pages": "2"
    - }
  - }
* }

Parsing one of these files is different from the summary files. Each section is divided into a thead section and a tbody section
The thead section only identifies what is to follow in the tbody section for example: <th class='table-name' colspan="4">Business Details ... </th> and does not contain headers for the actual tbody content.
Instead, the tbody section is made up of tr statements each contain pairs of td statements, odd one containing element name, and even element value.
The filings section does not conform to this structure. Instead, it has a true heading contained in the thead statement, followed by a tbody statement containing one or more tr statements, each containing as many td statements as there are th statements in the thead section.

There are 4 parts to each Business:
* Business Details
* Principals Details
* Agent Summary
* Filings Details

### The process:
***
### Setup

* create variables and lists used in __init__ method
* load CustomerMaster.json into self.BusinessInfo dictionary
* create detail download list (self.download_list) containing filename, url pairs for concurrent.futures download

### Download

method: download_detail() - calls fetch_url using all 4 cpu cores (change max_workers= #cores +1 for your processor)
Files are cached, so this process can be interrupted and started again at a leater time without data loss.
With about 400,000 pages for entire state, Total download time can be as long as 55.5 hours, though I have experienced much faster times in recent runs (I expect someone added an index to Db).

### Parse detail files

method: parse_detail()
* For each file in self.filelist:
  - Extract BusinessId from stem of filename.
  - Read existing Business data from CompanyMaster (BusinessInfo dictionary)
  - Read file and convert contents to soup
  - Get node from BusinessInfo dictionary, if not found (shouldn't be the case, but possible if some files manually downloaded) add to mising link list/

  - Now there are 4 sections of the file that need to be parsed, Business Details, Principal info agent info and filing info.
    All are dispatched the same way:
    
    - try:
        - discard = self.current_node['BusinessDetail']
    - except KeyError:
        - bnode = self.cd.add_node(self.current_node, 'BusinessDetail')
        = self.parse_business_details(soup, bnode)

  - This is trick code used to bypass already loaded nodes.
    the try will fail if the node doesn't exist in the dictionary, causing a new node to be created, and then populated with a call to method parse_business_details(soup, bnode)
    Note that the node is named differently for each section, bnode, pnode, anode, or fnode (1st letter same as 1st letter of section name (bnode = Business Details), 
    so when looking at code, you know which section of data you ar edealing with.
  - After all files have been processed (400,000) for full boat, save missing list to /data/json (can be use later to retry for missing files).

### parse_business_details(self, soup, bnode)
* Parser for Business Details html
  - Find table tag with id = 'detail-table'
  - Find all 'tr' tags.
  - for each tr tag:
    - Find all 'td' tags These are arranged as pairs, title in odd td's, value in even td's.
    - set pair flag to odd
    - If an element is not present, there will be a pair of empty td cells  this is what the skipeven flag is used for, usually I will provide
      empty node with proper title and value of unspecified.
    - For each td:
      - If odd:
        - extract title, fills some empty tag conditions, and replaces special characters (that will give SQL inserts the fits).
        - Set pair flag to even
      - If even:
        - if empty element, set value to 'Unspecified'
        - There is a duplicate BusinessId tag in this section, bypass it when found as it is redundit.
        - Add cell to dictionary
        - set pair flag to odd

### parse_principals(self, soup, pnode)
* Parser for Principal information
  - The html for principals is different from Business Details. Rather that use the odd, even td approach, it uses a header to get title information.
  - Find table tag with id = 'principals'
  - Find 'tr' tag in child node 'thead' (only one)
    - find all 'th' tags
      - for each, extract text, replace special characters and save in pheader list
  - find all 'tr' tags in child node 'tbody' (many)
    - for each 'tr' node:
      - find all 'td' tags
        - if 'td' element is empty, set value to 'Unspecified'
        - Otherwise extract text value
        - save cell to dictionary.

### parse_agents(self, soup, anode)
* parser for agent information
  This is the same format as principals, only difference here is the 'id' of the table element
  - Find table tag with id = 'agents'
  - remainder same as for parse_principals method

### parse_filings(self, soup, fnode)
* Parser for company filings
  This table is similar to principals and agents, with the exception that there may be None, or many.
  To compensate for this, each filing will be given a separate node in the dictionary, with a generated sequence number as key.
  - Find table tag with id = 'filings'
  - Header extraction same as for principals and agents.
  - tbody section has a separate tr for each filing, and contains the following elements:
    - Filing ID
    - Filing Type
    - Date of Filing (MM/DD/YYYY)
    - Volume Type
    - Volume
    - Start Page
    - Pages #
  - Parsing is the same as principals and agents with the addition of a sequence Node.
  
### Save results

rewrite CustomerMaster.json file with new iformation.

In [ ]:
# AddEntityDetail.py

import BusinessPaths
from pathlib import Path
from bs4 import BeautifulSoup
import PrettifyPage
import CreateDict
import json
import concurrent.futures
import os
import requests
import time
import sys

# use Id0667703.html for testing, has all fields

class AddEntityDetail:
    def __init__(self):
        self.bpath = BusinessPaths.BusinessPaths()
        self.pp = PrettifyPage.PrettifyPage()
        self.cd = CreateDict.CreateDict()

        self.header_replacepairs = [ ('(', ''), (')', ''), ('/', ''), (' ', ''),
            ('MMDDYYYY', '') ]

        self.missing = []
        self.BusinessInfo = {}
        self.new_business_info = {}
        self.current_node = {}
        self.download_list = []
        self.filelist = []
        self.filecount = 0

        self.add_entity_detail()
    
    def load_business_info(self):
        with self.bpath.company_master_json.open() as fp:
            self.BusinessInfo = json.load(fp)
        
        for BusinessId in self.BusinessInfo.keys():
            url = self.BusinessInfo[BusinessId]['DetailUrl']
            filename = Path(self.BusinessInfo[BusinessId]['Filename'])
            self.download_list.append([filename, url])
            self.filecount += 1
        self.download_list.sort()
        # self.cd.display_dict(self.BusinessInfo)

    def add_entity_detail(self):        
        self.load_business_info()
        # self.download_detail()
        self.parse_detail()
        self.save_as_json()

    def download_detail(self):
        print('starting download')
        countdown = self.filecount

        with concurrent.futures.ProcessPoolExecutor(max_workers=5) as executor:
            detail_info = [ executor.submit(self.fetch_url, filename, url) for filename, url in self.download_list if not filename.exists() ]

            for future in concurrent.futures.as_completed(detail_info):
                countdown -= 1
                print(f'countdown: {countdown}')
                filename = future.result()

    def fetch_url(self, filename, url):
        print(f'fetching {filename.name}')
        response = requests.get(url)
        if response.status_code == 200:
            # time.sleep(.25)
            with filename.open('wb') as fp:                
                fp.write(response.content)
        else:
            print(f"can't download: {url}")
            return False
        return filename

    def create_file_list(self):
        self.filelist = [filename for filename in self.bpath.idpath.iterdir()
            if filename.is_file and filename.stem.startswith('Id') ]
        self.filelist.sort()

    def parse_detail(self):
        self.create_file_list()

        for filename in self.filelist:
            print(f'processing {filename.name}')

            # Get existing dictionary node
            business_id = filename.stem[2:]
            try:
                self.current_node = self.BusinessInfo[business_id]
            except KeyError:
                self.missing.append(os.fspath(filename))
                continue

            # read file
            with filename.open() as fp:
                page = fp.read()
            soup = BeautifulSoup(page, 'lxml')

            # Add Business Detail
            try:
                discard = self.current_node['BusinessDetail']
            except KeyError:
                bnode = self.cd.add_node(self.current_node, 'BusinessDetail')
                self.parse_business_details(soup, bnode)

            # Add Principals Detail
            try:
                discard = self.current_node['PrincipalsDetail']
            except KeyError:
                pnode = self.cd.add_node(self.current_node, 'PrincipalsDetail')
                self.parse_principals(soup, pnode)

            # Add Agent Detail
            try:
                discard = self.current_node['AgentDetail']
            except KeyError:
                anode = self.cd.add_node(self.current_node, 'AgentDetail')
                self.parse_agents(soup, anode)

            # Add Filings Detail
            try:
                discard = self.current_node['FilingsDetail']
            except KeyError:
                fnode = self.cd.add_node(self.current_node, 'FilingsDetail')
                self.parse_filings(soup, fnode)
     
        missingfiles = self.bpath.jsonpath / 'missing.json'
        with missingfiles.open('w') as fp:
            json.dump(self.missing, fp)
            # verify
            # self.cd.display_dict(self.current_node)
            # print(self.current_node)

    def parse_business_details(self, soup, bnode):
        table = soup.find('table', {'class': 'detail-table'})
        trs = table.tbody.find_all('tr')
        for n, tr in enumerate(trs):            
            tds = tr.find_all('td')            
            odd = True
            skipeven = False
            for n1, td in enumerate(tds):
                # print(f'\n======================== tr_{n}, tr_ {n1} ========================')
                # print(f'{self.pp.prettify(td, 2)}')
                if skipeven:
                    skipeven = False
                    continue
                if odd:
                    if not len(td):
                        if n == 2 and n1 == 2:
                            title = 'BusinessType'
                            value = 'Unspecified'
                            self.cd.add_cell(bnode, title, value)
                        elif n == 4 and n1 == 2:
                            title = 'Unused'
                            value = 'Unspecified'
                            self.cd.add_cell(bnode, title, value)
                        skipeven = True
                        continue
                    title = td.text.strip()
                    if title[-1] == ':':
                        title = title[:-1]
                    title = title.replace('/', '')
                    title = title.replace(' ', '')
                    odd = False
                else:
                    if len(td):
                        value = td.text.strip()
                    else:
                        value = 'Unspecified'
                    # Already have business id, don't need twice
                    if title == 'BusinessID':
                        odd = True
                        continue
                    self.cd.add_cell(bnode, title, value)
                    odd = True

    def parse_principals(self, soup, pnode):
        # Get header
        principals = soup.find('table', {'id': 'principals'})
        if principals:
            phead = principals.thead.find('tr')
            pheader = []
            ths = phead.find_all('th')
            for th in ths:
                item = th.text.strip()
                for a, b in self.header_replacepairs:
                    item = item.replace(a, b)
                pheader.append(item)

            trs = principals.tbody.find_all('tr')
            for tr in trs:
                tds = tr.find_all('td')
                for n1, td in enumerate(tds):
                    if len(td):
                        self.cd.add_cell(pnode, pheader[n1], td.text.strip())
                    else:
                        self.cd.add_cell(pnode, pheader[n1], 'Unspecified')

    def parse_agents(self, soup, anode):
        agents = soup.find('table', {'id': 'agents'})
        if agents:
            aheader = []
            ahead = agents.thead.find('tr')
            ths = ahead.find_all('th')
            for th in ths:
                item = th.text.strip()
                for a, b in self.header_replacepairs:
                    item = item.replace(a, b)
                aheader.append(item)

            trs = agents.tbody.find_all('tr')
            for tr in trs:
                tds = tr.find_all('td')
                for n1, td in enumerate(tds):
                    if len(td):
                        self.cd.add_cell(anode, aheader[n1], td.text.strip())
                    else:
                        self.cd.add_cell(anode, aheader[n1], 'Unspecified')

    def parse_filings(self, soup, fnode):
            filings = soup.find('table', {'id': 'filings'})
            if filings:
                fheader = []
                fhead = filings.thead.find('tr')
                ths = fhead.find_all('th')
                for th in ths:
                    title = th.text.strip()
                    if '#' in title:
                        title = title[:-2]
                    for a, b in self.header_replacepairs:
                        title = title.replace(a, b)
                    fheader.append(title)
                
                trs = filings.find_all('tr')
                seq = 1
                for tr in trs:
                    tds = tr.find_all('td')
                    fitem = self.cd.add_node(fnode, str(seq))
                    for n1, td in enumerate(tds):
                        if len(td):
                            self.cd.add_cell(fitem, fheader[n1], td.text.strip())
                        else:
                            self.cd.add_cell(fitem, fheader[n1], 'Unspecified')
                    seq += 1
    
    def save_as_json(self):
        with self.bpath.company_master_json.open('w') as fp:
            json.dump(self.BusinessInfo, fp)

if __name__ == '__main__':
    AddEntityDetail()

***
## Database creation and load

There are two modules involved
Createtables.py and CreateDatabase.py


***
### CreateTables.py

This is a simple module that will create a new sqlite3 (can be simply modified for PostgreSQL, Oracle, Sybase or others)
It also builds Insert Queries when passed table name and a list of values (from called module)

Explain on the fly

In [ ]:
# CreateTables.py

import BusinessPaths
import sqlite3
import sys


class CreateTables:
    def __init__(self):
        self.bpath = BusinessPaths.BusinessPaths()

        self.insert_statements = {}
        self.dbcon = None
        self.dbcur = None

    def db_connect(self):
        try:
            self.dbcon = sqlite3.connect(self.bpath.CompanyMasterDb)
            self.dbcur = self.dbcon.cursor()
        except sqlite3.Error as e:
            print(e)

    def create_tables(self):
        self.db_connect()
        company = [ 
            'BusinessId', 'BusinessName', 'DetailURL', 'Filename', 'DateFormed',
            'Status', 'Address', 'CityState', 'PrincipalNames', 'AgentNames'
        ] 
        self.insert_statements['Company'] = self.create_table(company, 'Company')

        details = [
            'BusinessId', 'BusinessName', 'CitizenshipStateInc', 'LatestReport',
            'BusinessAddress', 'BusinessType', 'MailingAddress', 'BusinessStatus',
            'DateIncRegistration', 'Unused'
        ]
        self.insert_statements['Details'] = self.create_table(
            details, 'Details')

        # need to assign principalId
        principals = [
            'BusinessId', 'NameTitle', 'BusinessAddress', 'ResidenceAddress'
        ]
        self.insert_statements['Principals'] = self.create_table(
            principals, 'Principals')

        agents = [
            'BusinessId', 'AgentName', 'AgentBusinessAddress',
            'AgentResidenceAddress'
        ]
        self.insert_statements['Agents'] = self.create_table(
            agents, 'Agents')

        filings = [
            'BusinessId', 'SeqNo', 'FilingId', 'FilingType', 'DateofFiling',
            'VolumeType', 'Volume', 'StartPage', 'Pages'
        ]
        self.insert_statements['Filings'] = self.create_table(
            filings, 'Filings')

    def create_table(self, header, tablename='tname'):
        """
        Create CorpTable from header record of self.bpath.corporation_master
        """
        qmarks = (f"?, " * len(header))[:-2]
        base_insert = f"INSERT INTO {tablename} VALUES "
        columns = ', '.join(header)
        sqlstr = f'DROP TABLE IF EXISTS {tablename};'
        self.dbcur.execute(sqlstr)
        sqlstr = f'CREATE TABLE IF NOT EXISTS {tablename} ({columns});'
        # print(sqlstr)
        self.dbcur.execute(sqlstr)
        self.db_commit()
        return base_insert

    def insert_data(self, tablename, columns):
        # print(f'\n{tablename}: {columns}')
        dbcolumns = None
        try:
            dbcolumns = '('
            for item in columns:
                dbcolumns = f"{dbcolumns}'{item}', "
            dbcolumns = f"{dbcolumns[:-2]});"
            sqlstr = f'{self.insert_statements[tablename]}{dbcolumns}'
            # print(f'\n{tablename}: {sqlstr}')
            self.dbcur.execute(sqlstr)
        except sqlite3.OperationalError:
            print(f'OperationalError:\n{sqlstr}')
            sys.exit(0)
        except sqlite3.IntegrityError:
            print(f'IntegrityError:\n{sqlstr}')
            sys.exit(0)


    def db_close(self, rollback=False):
        if rollback:
            self.dbcon.rollback()
        else:
            self.dbcon.commit()
        self.dbcon.close()

    def db_commit(self):
        self.dbcon.commit()

if __name__ == '__main__':
    ct = CreateTables()
    ct.create_tables()


***
### CreateDatabase.py

This module creates the CompanyMaster.db Database in /data/database.

In [ ]:
# CreateDatabase.py

import BusinessPaths
import CreateTables
import sqlite3
import json
import sys


class CreateDatabase:
    def __init__(self):
        self.bpath = BusinessPaths.BusinessPaths()
        self.cretab = CreateTables.CreateTables()

        with self.bpath.company_master_json.open() as fp:
            self.master = json.load(fp)

        self.dbcon = None
        self.dbcur = None

        self.CompanyMain = {}
        self.Detail = {}
        self.Principals = {}
        self.Agents = {}
        self.Filings = {}

        self.insert_statements = {}

        self.db_connect()
        self.makedb()
        self.db_close()

    def db_connect(self):
        try:
            self.dbcon = sqlite3.connect(self.bpath.CompanyMasterDb)
            self.dbcur = self.dbcon.cursor()
        except sqlite3.Error as e:
            print(e)

    def db_close(self, rollback=False):
        if rollback:
            self.dbcon.rollback()
        else:
            self.dbcon.commit()
        self.dbcon.close()

    def db_commit(self):
        self.dbcon.commit()

    def insert_business_id(self, business_id, oldvalue):
        newvalue = {}
        newvalue['BusinessId'] = business_id
        for key, dvalue in oldvalue.items():
            newvalue[key] = dvalue
        return newvalue

    def split_dict(self):
        if not self.bpath.company_main.exists():
            for business_id, details in self.master.items():
                firstcompany = True
                print(f'Business_id: {business_id}')
                for key, value in details.items():
                    if key == 'BusinessDetail':
                        if len(value):
                            newvalue = self.insert_business_id(business_id, value)
                            self.Detail[business_id] = newvalue
                    elif key == 'PrincipalsDetail':
                        if len(value):
                            newvalue = self.insert_business_id(business_id, value)
                            self.Principals[business_id] = newvalue
                    elif key == 'AgentDetail':
                        if len(value):
                            newvalue = self.insert_business_id(business_id, value)
                            self.Agents[business_id] = newvalue
                    elif key == 'FilingsDetail':
                        if len(value):
                            newvalue = self.insert_business_id(business_id, value)
                            self.Filings[business_id] = newvalue
                    else:
                        if firstcompany:
                            cnode = self.CompanyMain[business_id] = {}
                            firstcompany = False
                        cnode[key] = value
            self.save_split_files()
        else:
            self.load_split_files()
    
    def save_split_files(self):
        with self.bpath.company_main.open('w') as fp:
            json.dump(self.CompanyMain, fp)

        with self.bpath.company_detail.open('w') as fp:
            json.dump(self.Detail, fp)

        with self.bpath.company_principals.open('w') as fp:
            json.dump(self.Principals, fp)

        with self.bpath.company_agents.open('w') as fp:
            json.dump(self.Agents, fp)

        with self.bpath.company_filings.open('w') as fp:
            json.dump(self.Filings, fp)

    def load_split_files(self):
        with self.bpath.company_main.open() as fp:
            self.CompanyMain = json.load(fp)

        with self.bpath.company_detail.open() as fp:
            self.Detail = json.load(fp)

        with self.bpath.company_principals.open() as fp:
            self.Principals = json.load(fp)

        with self.bpath.company_agents.open() as fp:
            self.Agents = json.load(fp)

        with self.bpath.company_filings.open() as fp:
            self.Filings = json.load(fp)

    def makedb(self):
        self.cretab.create_tables()
        self.split_dict()

        self.insert_data(self.CompanyMain, 'Company')
        self.insert_data(self.Detail, 'Details')
        self.insert_data(self.Principals, 'Principals')
        self.insert_data(self.Agents, 'Agents')
        self.insert_data(self.Filings, 'Filings')

    def insert_data(self, data_dict, tablename):
        print(f'Loading {tablename} table')
        self.base_insert = f"INSERT INTO {tablename} VALUES "        
        keys = list(data_dict.keys())
        for key in keys:
            try:
                data = data_dict[key]
                # print(f'data: {data}')
                columns = f"("
                for item in data:
                    value = data_dict[key][item]
                    if not len(value):
                        continue
                    if isinstance(value, dict):
                        columns = f"{columns}'{item}', "
                        for key1, subitem in value.items():
                            subitem = subitem.replace("'", "''")
                            columns = f"{columns}'{subitem}', "
                        break
                    value = value.replace("'", "''")
                    columns = f"{columns}'{value}', "
                columns = f"{columns[:-2]});"
                sqlstr = f'{self.base_insert}{columns}'
                # print(f'sqlstr: {sqlstr}')
                self.dbcon.execute(sqlstr)
            except sqlite3.OperationalError:
                print(f'sqlite3.OperationalError\ndata: {data}\nsqlstr: {sqlstr}')
                sys.exit(0)
            except AttributeError:
                print(f'AttributeError\nkey: {key}, item: {item}, value: {value}, data: {data}')
                sys.exit(0)
        self.db_commit()


if __name__ == '__main__':
    CreateDatabase()


# How to backup data including all 18 thousand summary pages and 400 thousand detail pages:
* Use rsync which is a quantum leap over cp in speed (FYI: includes using over network)
* get a list: 
  - find ./Idfiles/ -name *.html > ./tmp/Idlist.txt
  - run from makerProjectApril2019 directory
    - rsync -avz ./data/ backup_directory/data/